In [159]:
import os
from rosemary import jpt_in_notebook, jpt_setup; jpt_setup()
from llm.submit import submit_job, multiline_to_singleline, shell_scripts_template_slurm

log_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/'
nodes = 1; gpus=6
use_lora = False
text_pooling_type = 'meanpool'
grad_randproj_components = 2048
save_grad_embeddings = False

test_run = 0
test_run = bool(test_run)

#### auto-regressive model, compute loss/embedding/grad
# model_name = 'llama-7b'; model_name_or_path = '../results/baselines/huggyllama/llama-7b'; max_seq_len = 2048
# model_name = 'llama2-7b'; model_name_or_path = '../results/baselines/NousResearch/Llama-2-7b-hf'; max_seq_len = 2048
# model_name = 'llama-7b_ft=hmv1'; model_name_or_path = '../results/ft1/llama-7b_humanmix'; max_seq_len = 2048
# model_name = 'pythia-1b-deduped'; model_name_or_path = '../results/baselines/EleutherAI/pythia-1b'; max_seq_len = 2048
model_name = 'pythia-410M-deduped'; model_name_or_path = 'EleutherAI/pythia-410M-deduped'; max_seq_len = 2048
# model_name = 'mistral-7b'; model_name_or_path = '../results/baselines/mistralai/Mistral-7B-v0.1'; max_seq_len = 2048
# model_name = 'mistral-7b+ultrachat200kv1'; model_name_or_path = '../results/ft1_ep=2/mistral-7b_ultrachat200k_beforesplitlongconv'; max_seq_len = 2048
# model_name = 'llama-7b+lima'; model_name_or_path = '../results/oi2/llama-7b_lima_ep=5'; max_seq_len = 2048
# model_name = 'codellama-7b'; model_name_or_path = '../results/baselines/codellama/CodeLlama-7b-hf'; max_seq_len = 2048

compute_loss = True; compute_grad = True; encode_fn_type = 'sft' # 'sft'
# use_lora = True;  lora_rank = 256; lora_alpha = lora_rank; grad_randproj_components = 2048; add_rsum = True
use_lora = False; lora_rank = 256; lora_alpha = lora_rank; grad_randproj_components = 8192; add_rsum = False
####

# # encoder-based model, for computing embedding only
# model_name = 'all-mpnet-base-v2'; model_name_or_path = '../results/baselines/sentence-transformers/all-mpnet-base-v2'; max_seq_len = 512; text_pooling_type = 'meanpool'
# # model_name = 'bge-large-en-v1.5'; model_name_or_path = '../results/baselines/BAAI/bge-large-en-v1.5'; max_seq_len = 128; text_pooling_type = 'cls'

# compute_loss = False; compute_grad = False; encode_fn_type = 'input'
######

##### 
# datasets = ['open_orca_slim']; nodes=5; gpus=6; cpu_mem=512
# datasets = ['sharegptv2']; nodes=5; gpus=6; cpu_mem=512
# datasets = ['tulu_v2']; nodes=10; gpu=6; cpu_mem=512
# datasets = ['stanford_alpaca']; nodes=5; gpu=6; cpu_mem=512
# datasets = ['flan_v2', 'oasst1']; nodes=1; gpu=6; cpu_mem=512
# datasets = ['dolly']; nodes=1; gpu=6; cpu_mem=512

#####
# tulu_v1_mix:
# datasets = ['flan_v2', 'cot', 'dolly', 'oasst1', 'gpt4_alpaca', 'code_alpaca', 'sharegpt']; nodes = 5; gpu=6; cpu_mem = 512
# datasets = ['tulu_v1_human_mix', 'tulu_v1_mix']; nodes = 1; gpus=1; cpu_mem = 64
# tulu_v2_mix:
# datasets = ['tulu_v2']; nodes=1; gpus=1; cpu_mem=512
# datasets = ['cot', 'dolly', 'flan_v2', 'lima', 'oasst1', 'gpt4_alpaca', 'code_alpaca']; nodes = 3; gpus=6; cpu_mem = 512
# datasets = ['wizardlm']; nodes = 3; gpus=6; cpu_mem=512
# datasets = ['sharegpt', 'ultrachat']; nodes = 5; gpus=6; cpu_mem = 512
# datasets = ['ultrachat15']; nodes = 3; gpus=6; cpu_mem = 512
# datasets = [f'ultrachat15_{i}' for i in [0, 2]]; nodes = 5; gpus=6; cpu_mem = 512
# datasets = ['starcoder_commentinstr_cleaned',
#             'starcoder_commentinstr',]; nodes=1; gpus=6; cpu_mem=512
# datasets = [
#     'starcoder_commentinstrv2', # cleaned version
# #     'starcoder_commentinstrv3',
# ]; nodes = 3; gpus = 6; cpu_mem=512

# datasets = ['starcoder_commentinstrv5']; nodes = 3; gpus = 6; cpu_mem=512

# datasets = ['tulu_v2_human_mix', 'tulu_v2_mix']; nodes = 1; gpus=1; cpu_mem = 64
# datasets = ['flan_v2']; nodes = 1; gpus=6; cpu_mem = 512
# datasets = ['flan2022_1m']; nodes = 5; gpus=6; cpu_mem = 512 # if not compute grad embeddings
# datasets = ['flan2022_1m']; nodes = 15; gpus=6; cpu_mem = 512 # if do compute grad embeddings
# datasets = ['flan2022v2_1m']; nodes = 5; gpus=6; cpu_mem = 512

# # for testing
datasets = ['lima']; nodes = 1; gpus=6; cpu_mem = 512; save_grad_embeddings = True

save_dir = (f"/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/"
            f"model_outputs/{encode_fn_type}/{model_name}")
if use_lora:
    save_dir += f'+lora:r={lora_rank}:a={lora_alpha}'

use_dist = True if nodes*gpus>1 else False
for dataset in datasets:
    
    if use_dist:
        prefix = f"""
    torchrun --nnodes={nodes} --nproc_per_node={gpus} \
        --rdzv-id=$SLURM_JOB_ID --rdzv-backend=c10d --rdzv-endpoint=$RDZV_ENDPOINT \
        """
    else:
        prefix = 'python'
    
    cmd = f"""
        {prefix}
        note_llama_embeddings.py \
        --dataset={dataset} \
        --model_name_or_path={model_name_or_path} \
        {'--use_dist' if use_dist else ''} \
        --shuffle \
        {'--compute_loss' if compute_loss else ''} \
        {'--compute_grad' if compute_grad else ''} \
        {'--use_lora' if use_lora else ''} \
        {'--lora_rank='+str(lora_rank) if use_lora else ''} \
        {'--lora_alpha='+str(lora_alpha) if use_lora else ''} \
        --compute_grad_embeddings \
        {'--save_grad_embeddings' if save_grad_embeddings else ''} \
        --grad_randproj_components {grad_randproj_components} \
        --max_seq_len={max_seq_len} \
        --encode_fn_type={encode_fn_type} \
        --text_pooling_type={text_pooling_type} \
        {'--add_rsum' if add_rsum else ''} \
        --save_dir={save_dir} \
    """
    cmd = multiline_to_singleline(cmd)

    shell_scripts = shell_scripts_template_slurm.format(
        conda_env='open-instruct',
        cwd=os.getcwd(),
        cmd=cmd,
        log_dir=log_dir,
        save_dir=save_dir)
    out = submit_job(
        shell_scripts, 
        job_name=f'lm_outputs.{dataset}', 
        nodes=nodes,
        num_cpus=32,
        cpu_mem=cpu_mem,
        num_gpus=gpus,
        gpu_type='v100',
        test_run=test_run,
        job_duration=6,
    )
    print(cmd)
    if not test_run:
        print(out)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



Submiting job with:
{
    "job_name": "lm_outputs.lima",
    "nodes": 1,
    "num_cpus": 32,
    "cpu_mem": 512,
    "num_gpus": 6,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
torchrun --nnodes=1 --nproc_per_node=6 --rdzv-id=$SLURM_JOB_ID --rdzv-backend=c10d --rdzv-endpoint=$RDZV_ENDPOINT note_llama_embeddings.py --dataset=lima --model_name_or_path=EleutherAI/pythia-410M-deduped --use_dist --shuffle --compute_loss --compute_grad --compute_grad_embeddings --save_grad_embeddings --grad_randproj_components 8192 --max_seq_len=2048 --encode_fn_type=sft --text_pooling_type=meanpool --save_dir=/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/model_outputs/sft/pythia-410M-deduped
[{'args': 'sbatch --job-name=lm_outputs.lima --partition=el8 --nodes=1 --ntasks-per-node=1 --cpus-per-task=32 --mem=512GB --gres=gpu:6 --output=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/%J.out --time=6:0

In [8]:
!CUDA_VISIBLE_DEVICES=4 python note_llama_embeddings.py --dataset=open_orca_slim --model_name_or_path=../results/baselines/NousResearch/Llama-2-7b-hf --shuffle --compute_loss --compute_grad --use_lora --lora_rank=256 --lora_alpha=256 --compute_grad_embeddings --grad_randproj_components 2048 --max_seq_len=2048 --encode_fn_type=output --text_pooling_type=meanpool --save_dir=/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/model_outputs/output/llama2-7b+lora:r=256:a=256


[2023-12-14 23:25:33,239] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
rank/local_rank/world_size: 0/0/1

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:03<00:00,  1.76s/it]
/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in 


KeyboardInterrupt



In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()


if jpt_in_notebook():
    import os
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    os.environ['CUDA_VISIBLE_DEVICES'] = \
        os.environ['CUDA_VISIBLE_DEVICES'].split(',')[1] 
    # '0,1,2,3,4,5'
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


0,1
1
Wed Dec 27 12:25:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   30C    P0    38W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| 

In [2]:
from collections import defaultdict
from functools import partial
import os
import numpy as np
import time
import re
import random
import pickle
from tqdm import tqdm 
import pandas as pd
from sklearn.random_projection import SparseRandomProjection


import pyarrow # import before `torch`, `transformers`, `datasets`
import torch
from torch.utils.data import DataLoader

from datasets import load_dataset

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

from open_instruct.finetune_trainer import encode_with_prompt_completion_format, encode_with_messages_format
from note_llama_embeddings import (
    encode_just_one_role,
    sklearn_rp_mat_size,
    combine_lm_outputs_for_mixes, 
    datasets_shard_chunk_size, 
    compute_losses, 
    compute_grad_statistic, 
    compute_grad_norm,
    gather_grad_embeddings,
    print_trainable_parameters,
    get_grad_statistic_pattern,
)



[2023-12-27 12:25:48,141] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [141]:
test_run = True

save_grad_embeddings = False
dataset = 'flan_v2'
dataset = 'oasst1'
dataset = 'tulu_v1_mix'
dataset = 'lima'
dataset = 'ultrachat15_0'
dataset = 'wizardlm'
dataset = 'stanford_alpaca'
dataset = 'lima'; save_grad_embeddings = True

use_dist = False
shuffle = True
lora_rank = 256
lora_alpha = lora_rank
grad_randproj_components = 2048
encode_fn_type = 'input'  # input, output, sft
encode_fn_type = 'output'
max_seq_len = 2048
text_pooling_type = 'meanpool'
add_rsum = False

#####
# model_name = 'llama-7b'; model_name_or_path = '../results/baselines/huggyllama/llama-7b'
# model_name = 'pythia-1b-deduped'; model_name_or_path = 'EleutherAI/pythia-1b-deduped'
# model_name = 'pythia-70m-deduped'; model_name_or_path = 'EleutherAI/pythia-70m-deduped'
model_name = 'pythia-410M-deduped'; model_name_or_path = 'EleutherAI/pythia-410M-deduped'
# model_name = 'mistral-7b'; model_name_or_path = '../results/baselines/mistralai/Mistral-7B-v0.1'
compute_grad_embeddings = True; use_lora = True; compute_grad=True; compute_loss = True
compute_grad_embeddings = True; use_lora = False; compute_grad=True; compute_loss = True
#####

# #####
# model_name = 'all-mpnet-base-v2'; model_name_or_path = '../results/baselines/sentence-transformers/all-mpnet-base-v2'
# compute_grad_embeddings = False; use_lora = False; compute_grad = False; compute_loss = False
# #####


save_dir = f"model_outputs/{encode_fn_type}/{model_name}"
if not test_run:
    os.makedirs(save_dir, exist_ok=True)
    
print(save_dir)

model_outputs/output/pythia-410M-deduped


In [142]:
if dataset in ['tulu_v1_human_mix', 
               'tulu_v1_mix',
               'tulu_v2_human_mix',
               'tulu_v2_mix']:
    combine_lm_outputs_for_mixes(dataset, save_dir)

if use_dist:
    dist.init_process_group("gloo", timeout=datetime.timedelta(hours=6))
    world_size = dist.get_world_size()
    rank = dist.get_rank() # global rank
    local_rank = int(os.environ["LOCAL_RANK"])
else:
    rank = 0
    local_rank = 0
    world_size = 1

print(f'rank/local_rank/world_size: {rank}/{local_rank}/{world_size}\n')

device = f'cuda:{str(local_rank)}'

rank/local_rank/world_size: 0/0/1



In [143]:
if 'sentence-transformers' in model_name_or_path:
    from transformers import AutoModel
    model = AutoModel.from_pretrained(
        model_name_or_path,
        device_map=device)
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        device_map=device,
        torch_dtype=torch.float16)
    

In [144]:
if use_lora:
    if not compute_grad:
        raise ValueError('compute_grad must be True if use LoRA!')
    
    print(f'Initializing lora(r={lora_rank},a={lora_alpha})')
    # ensure the same initialization
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    
    if any(x in model_name_or_path.lower() for x in ['llama', 'mistral']):
        # # the following also applies lora to MLP layers.
        # target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
        target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj']
    elif 'pythia' in model_name_or_path:
        target_modules = ['query_key_value']
    else:
        raise ValueError(f'Define new `target_modules` for LoraConfig for {model_name_or_path}')

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        bias='none',
        r=lora_rank,
        lora_alpha=lora_alpha, 
        lora_dropout=0.,
        target_modules=target_modules,
    )
    
    model.enable_input_require_grads()
    model = get_peft_model(model, peft_config)
    
    ## don't need to compute gradient to `lora_A`, saves computation (i think) but not space.
    for param_name, param in model.named_parameters():
        if param.requires_grad and 'lora_A' in param_name:
            param.requires_grad = False

print_trainable_parameters(model)

trainable params: 405334016 || all params: 405334016 || trainable%: 100.00


In [145]:
if compute_grad:
    if any(x in model_name_or_path.lower() for x in ['llama', 'mistral']):
        # Computing full gradient for llama is computationally prohibitive.
        # Use gradient checkpointing to prevent oom issues.
        # Note gradient checkpointing is only applied when in training mode
        #     https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py#L908
        # So need to set `model.train()`. This is harmless because
        # llama's eval/train computation is exactly the same, since there's no dropout layer.
        model.gradient_checkpointing_enable()
        model.train()
else:
    model.eval()

In [146]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, use_fast=True)
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [147]:

processed_dir = '../data/processed'
if 'flan2022' in dataset:
    train_file = os.path.join(processed_dir, 'flan2022', f'{dataset}_data.jsonl')
elif 'ultrachat' in dataset:
    train_file = os.path.join(processed_dir, 'ultrachat', f'{dataset}_data.jsonl')
elif 'open_orca' in dataset:
    train_file = os.path.join(processed_dir, 'open_orca', f'{dataset}_data.jsonl')
elif 'sharegpt' in dataset:
    train_file = os.path.join(processed_dir, 'sharegpt', f'{dataset}_data.jsonl')
elif 'starcoder' in dataset:
    train_file = os.path.join(processed_dir, 'starcoder', f'{dataset}.jsonl')
else:
    train_file = os.path.join(processed_dir, dataset, f'{dataset}_data.jsonl')
assert(os.path.isfile(train_file))

if encode_fn_type in ['input', 'output']:
    encode_function = partial(
        encode_just_one_role,
        tokenizer=tokenizer,
        max_seq_length=max_seq_len,
        encode_fn_type=encode_fn_type,
        # add eos token to causal models, e.g., llama, since its not added by default.
        add_eos_token=False if any(y in model_name_or_path for y in ['mpnet', 'bge']) else True,
    )
elif encode_fn_type == 'sft':    
    encode_function = partial(
        encode_with_messages_format,
        tokenizer=tokenizer,
        max_seq_length=max_seq_len,
        add_eos_token=False,
    )
else:
    raise ValueError(f'encode_fn_type={encode_fn_type} not implemented.')

In [148]:
if rank == 0:
    raw_datasets = load_dataset("json", data_files={'train': train_file})
    if test_run:
        raw_datasets['train'] = raw_datasets['train'].select(range(1000))
    # print(f"{dataset} dataset length = {len(raw_datasets['train'])}")
    lm_datasets = raw_datasets.map(
        encode_function, batched=False, num_proc=16,
        desc="Tokenizing and reformatting instruction data")
if use_dist:
    dist.barrier()
if rank!= 0:
    raw_datasets = load_dataset("json", data_files={'train': train_file})
    if test_run:
        raw_datasets['train'] = raw_datasets['train'].select(range(1000))
    # print(f"{dataset} dataset length = {len(raw_datasets['train'])}")
    lm_datasets = raw_datasets.map(
        encode_function, batched=False, num_proc=16,
        desc="Tokenizing and reformatting instruction data")

train_dataset = lm_datasets['train']
train_dataset.set_format(
    type="torch",
    output_all_columns=False,
    columns=['input_ids', 'labels', 'attention_mask'])
if shuffle:
    random.seed(0)
    shuffle_inds = list(range(len(train_dataset)))
    random.shuffle(shuffle_inds)
    reverse_shuffle_inds = [(i, ind) for i, ind in enumerate(shuffle_inds)]
    reverse_shuffle_inds = sorted(reverse_shuffle_inds, key=lambda x: x[1])
    reverse_shuffle_inds = [x[0] for x in reverse_shuffle_inds]
    train_dataset = train_dataset.select(shuffle_inds)
train_dataset_chunk_sizes = [datasets_shard_chunk_size(len(train_dataset), num_shards=world_size, index=i) 
            for i in range(world_size)]
train_dataset = train_dataset.shard(
    num_shards=world_size, 
    index=rank,
    contiguous=True)
loader = DataLoader(train_dataset, shuffle=False, batch_size=1, pin_memory=True) 

Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-1ca1bac0eed76345/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-1ca1bac0eed76345/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-4dcb5011d833d828_*_of_00016.arrow


In [149]:

grad_statistic_patterns = get_grad_statistic_pattern(model_name_or_path, use_lora)
grad_statistic_patterns


{'all': '.*',
 'qkv': '\\bquery_key_value\\.weight\\b',
 'mlp': '\\bmlp\\..*?\\.weight\\b',
 'last': '\\bembed_out\\.weight\\b'}

In [150]:
from note_llama_embeddings import mean_pooling

if compute_grad_embeddings:
    rps = {}
    for k in grad_statistic_patterns.keys():
        rps[k] = SparseRandomProjection(n_components=grad_randproj_components, random_state=0)

output = defaultdict(list)
for i, batch in tqdm(enumerate(loader), disable=rank!=0, total=len(loader)):
    batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
    
    if compute_grad:
        outputs = model(**batch, output_hidden_states=True, use_cache=False)
        model.zero_grad()
        outputs['loss'].backward()
    else:
        with torch.inference_mode():
            outputs = model(**batch, output_hidden_states=True)
    
    # (bsz, seq_len, hidden_size) -> (bsz, hidden_size)
    last_hidden_state = outputs['hidden_states'][-1]
    if text_pooling_type == 'meanpool':
        text_embedding = mean_pooling(last_hidden_state, batch['attention_mask'])
    elif text_pooling_type == 'cls':
        text_embedding = last_hidden_state[:, 0]
    else:
        raise ValueError(f'text_pooling_type={text_pooling_type} not supported.')
    output['text_embedding'].append(text_embedding.to(torch.float32).detach().cpu())

    if compute_loss:
        # average of output token log probs
        if 'loss' in outputs:
            output['log_prob'].append(-outputs['loss'].detach().cpu())

        # el2n scores
        losses = compute_losses(outputs['logits'], batch['labels'])
        for k in ['el2n_agg=mean', 'el2n_agg=l2n', 'logit_margin']:
            output[k].append(losses[k].detach().cpu())
            
    ## gradient statistic
    if compute_grad:
        grad_statistics = compute_grad_statistic(model, grad_statistic_patterns)
        for k, v in grad_statistics.items():
            output[f'grad_{k}'].append(v.detach().cpu())
            
        if compute_grad_embeddings:
            grad_embeddings = gather_grad_embeddings(
                model,
                {k: v for k, v in grad_statistic_patterns.items() if k in ['qkv', 'loraB']},
                stacked=True,
                add_rsum=add_rsum,
            )
            if save_grad_embeddings:
                for k, v in grad_embeddings.items():
                    output[f'grad_{k}'].append(v)
            if i==0:
                rps = {}
                for k, v in grad_embeddings.items():
                    t0 = time.time()
                    if 'rsum' not in k:
                        rps[k] = SparseRandomProjection(n_components=grad_randproj_components, random_state=0)
                    else:
                        rps[k] = GaussianRandomProjection(n_components=grad_randproj_components, random_state=0)
                    print(f"Fitting random projection for {k} ({v.size} -> {grad_randproj_components})")
                    rps[k] = rps[k].fit(v[np.newaxis,...])
                    print(f"Fitting random projection in {time.time() - t0:0.3f}s "
                        f"with random matrix size {sklearn_rp_mat_size(rps[k]) / 1e6:0.3f} MB")
                    print(f'Log statistics of projection matrix for {k} to ensure same initialization cross procs:\n'
                        f"{np.mean(rps[k].components_ != 0)}, {np.max(rps[k].components_)}, {np.mean(rps[k].components_[0])}")
            for k in grad_embeddings.keys():
                rp = rps[k]
                g = grad_embeddings[k]
                output[f'grad_rp_{k}'].append(rp.transform(g[np.newaxis,...]).squeeze())
    i += 1
    if i == 1:
        break


  0%|          | 0/1000 [00:00<?, ?it/s]

Fitting random projection for qkv (75497472 -> 2048)
Fitting random projection in 145.680s with random matrix size 142.364 MB
Log statistics of projection matrix for qkv to ensure same initialization cross procs:
0.00011509272735564052, 2.059767246246338, -3.1920640140015166e-06


  0%|          | 0/1000 [02:26<?, ?it/s]


In [151]:
for k, v in output.items():
    if isinstance(v, torch.Tensor):
        output[k] = torch.vstack(v).to(torch.float32).numpy()
    else:
        output[k] = np.vstack(v)


print(f'[local_rank/global={local_rank}/{rank}] '
      f'output={[(k, v.shape, v.dtype) for k, v in output.items()]}')

[local_rank/global=0/0] output=[('text_embedding', (1, 1024), dtype('float32')), ('log_prob', (1, 1), dtype('float16')), ('el2n_agg=mean', (1, 1), dtype('float16')), ('el2n_agg=l2n', (1, 1), dtype('float16')), ('logit_margin', (1, 1), dtype('float16')), ('grad_all_l2n', (1, 1), dtype('float32')), ('grad_qkv_l2n', (1, 1), dtype('float32')), ('grad_mlp_l2n', (1, 1), dtype('float32')), ('grad_last_l2n', (1, 1), dtype('float32')), ('grad_qkv', (1, 75497472), dtype('float32')), ('grad_rp_qkv', (1, 2048), dtype('float32'))]


In [ ]:

save_path = 'model_outputs/llama-7b+lora:r=256:a=256/lima.pkl'
with open(save_path, 'rb') as f:
    x = pickle.load(f)
x

In [ ]:
t0 = time.time()

for k in grad_embeddings.keys():
    rp = rps[k]
    g = grad_embeddings[k]
    rp.transform(g[np.newaxis,...]).squeeze()
print(time.time()-t0)

In [ ]:
save_path = f'note_llama_embeddings_{dataset}:{model_name}'+\
    (f'+lora(r={lora_rank},a={lora_alpha})' if use_lora else '')+'.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(output, f)
save_path

In [ ]:
import pickle
save_path = 'note_llama_embeddings_lima:pythia-160m-deduped.pkl'
with open(save_path, 'rb') as f:
    o1 = pickle.load(f)
save_path = 'note_llama_embeddings_lima:pythia-160m-deduped+lora(r=1,a=1).pkl'
save_path = 'note_llama_embeddings_lima:pythia-160m-deduped+lora(r=256,a=256).pkl'
with open(save_path, 'rb') as f:
    o2 = pickle.load(f)
    
output = {}
for k in ['grad_qkv', 'grad_last', 'grad_rp_qkv', 'grad_rp_last']:
    output[k] = o1[k]
for k in ['grad_loraB', 'grad_rp_loraB']:
    output[k] = o2[k]
print(output.keys())

In [1]:
import pickle
import os
from note_llama_embeddings import torch_cdist_chunked, plt_pair_of_dists


model_name = 'pythia-410M-deduped+lora:r=256:a=256'
dataset = 'lima'

Ds = {}
save_path = os.path.join('model_outputs', 'sft', model_name, f'{dataset}.pkl')
with open(save_path, 'rb') as f:
    o = pickle.load(f)
    for k in [x for x in o.keys() if x.startswith('grad') and 'l2n' not in x]:
        print(f'Computing distance for {k} of size {o[k].shape}')
        Ds[k] = torch_cdist_chunked(o[k], device='cuda', chunk_size='auto', mem_total=12)
    del o
    
save_path = os.path.join('model_outputs', 'sft', model_name, f'{dataset}_dist.pkl')
with open(save_path, 'wb') as f:
    pickle.dump(Ds, f)

[2023-12-27 15:45:52,609] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Computing distance for grad_loraB of size (1030, 18874368)
set chunk_size = 85 for (N, D) = (1030, 18874368)
Computing distance for grad_loraBrsum of size (1030, 6144)
set chunk_size = 1030 for (N, D) = (1030, 6144)
Computing distance for grad_loraBrsumcyc of size (1030, 6144)
set chunk_size = 1030 for (N, D) = (1030, 6144)
Computing distance for grad_rp_loraB of size (1030, 2048)
set chunk_size = 1030 for (N, D) = (1030, 2048)
Computing distance for grad_rp_loraBrsum of size (1030, 2048)
set chunk_size = 1030 for (N, D) = (1030, 2048)
Computing distance for grad_rp_loraBrsumcyc of size (1030, 2048)
set chunk_size = 1030 for (N, D) = (1030, 2048)


In [ ]:
Ds = {}
for k in [x for x in o.keys() if x.startswith('grad') and 'l2n' not in x]:
    print(f'Computing distance for {k} of size {o[k].shape}')
    Ds[k] = torch_cdist_chunked(o[k], device='cuda', chunk_size='auto', mem_total=12)


Computing distance for grad_qkv of size (1030, 75497472)
set chunk_size = 21 for (N, D) = (1030, 75497472)


In [ ]:

model_name = 'pythia-410M'
dataset = 'lima'
save_path = os.path.join('model_outputs', 'sft', model_name, f'{dataset}_dist.pkl')
with open(save_path, 'wb') as f:
    pickle.dump(Ds, f)

In [7]:

save_path = os.path.join('model_outputs', 'sft', 'pythia-410M-deduped', 'lima.pkl')
with open(save_path, 'rb') as f:
    o = pickle.load(f)

In [8]:

    for k in [x for x in o.keys() if x.startswith('grad') and 'l2n' not in x]:
        print(f'Computing distance for {k} of size {o[k].shape}')
        Ds[k] = torch_cdist_chunked(o[k], device='cuda', chunk_size='auto', mem_total=12)D

grad_qkv (1030, 75497472)
grad_rp_qkv (1030, 8192)


In [2]:

dataset = 'lima'; model_name = 'pythia-410M-deduped+lora:r=256:a=256'

for k in D.keys():
    if k == 'grad_qkv':
        continue
    fig, axs = plt_pair_of_dists(
        D['grad_qkv'], 
        dists[k]/(dists[k].mean()/dists['grad_qkv'].mean()), 
        n_components,
        use_hexbin=False)
    fig.suptitle(f'{dataset}:{model_name} grad_qkv vs. {k}')
    fig.tight_layout()


{'grad_loraB': array([[  0.48663786, 196.28995   , 116.134636  , ..., 114.07352   ,
         118.03757   , 200.18613   ],
        [196.28995   ,   1.5334805 , 202.33885   , ..., 213.8344    ,
         213.30598   , 257.0312    ],
        [116.134636  , 202.33885   ,   0.8244316 , ..., 136.88992   ,
         137.33588   , 204.2138    ],
        ...,
        [114.073494  , 213.83441   , 136.88991   , ...,   0.        ,
         145.2757    , 213.38147   ],
        [118.03756   , 213.30603   , 137.33586   , ..., 145.2757    ,
           0.        , 216.55844   ],
        [200.18614   , 257.03116   , 204.2138    , ..., 213.38147   ,
         216.55844   ,   0.        ]], dtype=float32),
 'grad_loraBrsum': array([[0.00000000e+00, 2.29321045e+02, 1.15307007e+02, ...,
         1.07796211e+02, 9.17635651e+01, 1.73650650e+02],
        [2.29321045e+02, 0.00000000e+00, 2.59919891e+02, ...,
         2.53984772e+02, 2.36575714e+02, 2.79136566e+02],
        [1.15307007e+02, 2.59919891e+02, 4.4194173

In [ ]:
import scipy
from note_llama_embeddings import torch_cdist
import time
from sklearn.datasets import load_digits
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.random_projection import SparseRandomProjection
import matplotlib.pyplot as plt

n_components = 2048
k = 'qkv'
# ['qkv', 'last', 'rp_qkv', 'rp_last', 'loraB', 'rp_loraB']

device = 'cuda'
data = output[f'grad_qkv']
projected_data = output[f'grad_rp_qkv']


dists = {}
ks = output.keys()
ks = ['grad_loraB', 'grad_rp_loraB']

for k in ks:
    if 'last' in k:
        continue
    data = output[k]
#     data = torch.from_numpy(data).to(torch.float32).to('cuda')
#     D = torch_cdist(data, device).ravel()
    D = scipy.spatial.distance.pdist(data)
#     valid_dist = np.logical_and(~np.isnan(dists), dists!=0)
#     dists = dists[valid_dist]
#     print(f'percent valid: {100*np.sum(valid_dist)/dists.size:.2f}')
    dists[k] = D


# t0 = time.time()
# rp = SparseRandomProjection(n_components=n_components)  
# projected_data = rp.fit_transform(output[f'grad_{k}'])
# print(
#     f"Projected in"
#     f"{time.time() - t0:0.3f}s"
# )
# if hasattr(rp, "components_"):
#     n_bytes = rp.components_.data.nbytes
#     n_bytes += rp.components_.indices.nbytes
#     print(f"Random matrix with size: {n_bytes / 1e6:0.3f} MB")

# projected_dists = torch_cdist(projected_data, device).ravel()
# del projected_data
# projected_dists = projected_dists[valid_dist]


In [160]:
from note_llama_embeddings import torch_cdist, plt_pair_of_dists

plt_pair_of_dists


<function note_llama_embeddings.plt_pair_of_dists(D1, D2, n_components, use_hexbin=True)>

In [ ]:
from time import time
from sklearn.datasets import load_digits
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.random_projection import SparseRandomProjection
import matplotlib.pyplot as plt
from note_llama_embeddings import torch_cdist, plt_pair_of_dists

dataset = 'lima'; model_name = 'pythia-160m-deduped+lora(rank=256,alpha=256)'

for k in dists.keys():
    if k == 'grad_qkv':
        continue
    fig, axs = plt_pair_of_dists(
        dists['grad_qkv'], 
        dists[k]/(dists[k].mean()/dists['grad_qkv'].mean()), 
        n_components,
        use_hexbin=False)
    fig.suptitle(f'{dataset}:{model_name} grad_qkv vs. {k}')
    fig.tight_layout()


In [ ]:
dists['grad_loraB'].mean()/dists['grad_qkv'].mean()

In [ ]:

param_names = []
grads = []
for param_name, param in model.named_parameters():
    if param.requires_grad and param.grad is not None:
        param_names.append(param_name)
        grads.append(param.grad.to(torch.float32))

data = []
for param_name, grad in zip(param_names, grads):
    data.append({
        'param_name': param_name,
        'shape': grad.shape,
        'numel': grad.numel(),
        'mean': grad.mean().detach().cpu().item(),
        'norm': grad.norm().detach().cpu().item(),
    })
        
        

df = pd.DataFrame(data)

# pattern = r'\bquery_key_value\.weight\b'
# # pattern = r'\bmlp\..*?\.weight\b'
pattern = r'\blora_B\b'

df = df[df['param_name'].apply(lambda x: True if re.search(pattern, x) else False)]
df = df.sort_values(['norm'], ascending=False)

df

In [ ]:
plt.hist(df['norm'].to_numpy())

In [ ]:
plt.hist(df['norm'].to_numpy())

In [ ]:
# output_fullgrad = output.copy()

In [ ]:

output_fullgrad.update({f'grad_loraB(r={lora_rank})_l2n': output['grad_loraB_l2n']})
# del output_fullgrad['grad_loraB_l2n']

In [ ]:
from rosemary import plt_kernel_matrix_one
import matplotlib.pyplot as plt
from scipy import stats

metrics_name = ['log_prob', 'el2n_agg=mean', 'el2n_agg=l2n', 'logit_margin']
metrics_name += [x for x in output_fullgrad if 'grad' in x and 'sum' not in x]
metrics = [output_fullgrad[k] for k in metrics_name]
any_isnan_mask = np.hstack([np.isnan(x) for x in metrics]).any(-1)
metrics = [x[~any_isnan_mask] for x in metrics]
N = len(metrics)
K = np.zeros((N, N))
for i, mi in enumerate(metrics):
    for j, mj in enumerate(metrics):
        s = stats.spearmanr(mi, mj)
        K[i,j] = s.statistic
        

fig, ax = plt.subplots(1,1,figsize=(8,8))
plt_kernel_matrix_one(
    fig, ax, K, title=f'Spearmanr ({model_name}:{dataset})', 
    vmin=-1, vmax=1, cmap='bwr', n_ticks=N, annotate=True)
ax.set_xticklabels(metrics_name, rotation=-45)
ax.set_yticklabels(metrics_name)

fig.tight_layout()